<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [10]:
%%capture
!pip install kaggle

In [11]:

%%capture
!pip install -U transformers[torch] datasets


%%capture
!pip install tensorboards

In [13]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

print("Upload the kaggle.json file")
# Ensure files.upload() is called in the same cell as its import
files.upload() # Calling files.upload() immediately after import
time.sleep(5)

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
time.sleep(5)

!kaggle datasets download -d dejolilandry/iemocapfullrelease
time.sleep(5)

with zipfile.ZipFile('iemocapfullrelease.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')
time.sleep(5)

!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)

!unzip /content/IEMOCAP_splits.zip
time.sleep(5)

Upload the kaggle.json file


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/dejolilandry/iemocapfullrelease
License(s): unknown
100% 11.5G/11.5G [02:25<00:00, 82.3MB/s]
100% 11.5G/11.5G [02:25<00:00, 85.1MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 10.6MB/s]
Archive:  /content/IEMOCAP_splits.zip
replace iemocap/session5/test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: o
error:  invalid response [o]
replace iemocap/session5/test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace iemocap/session5/train.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [17]:
import json
import os

# Define base path for sessions and the path prefix to replace
base_path = "/content/"
old_path_prefix = "/m/teamwork/t40511_asr/c/"

# List of sessions to iterate over
sessions = ["session1", "session2", "session3", "session4", "session5"]

# Process each session
for session in sessions:
    # Define paths to train and test JSON files
    train_json_path = os.path.join(base_path, session, "train.json")
    test_json_path = os.path.join(base_path, session, "test.json")

    # Function to update JSON file paths
    def update_file_paths(json_path):
        if os.path.exists(json_path):
            # Load JSON data
            with open(json_path, "r") as file:
                data = json.load(file)

            # Update each entry's 'wav' path
            for file_id, entry in data.items():
                if entry["wav"].startswith(old_path_prefix):
                    # Replace the old prefix with the new base path
                    entry["wav"] = entry["wav"].replace(old_path_prefix, base_path)

            # Save the updated JSON data
            with open(json_path, "w") as file:
                json.dump(data, file, indent=4)
            print(f"File paths updated successfully in {json_path}")
        else:
            print(f"{json_path} does not exist.")

    # Update train and test files
    update_file_paths(train_json_path)
    update_file_paths(test_json_path)


/content/session1/train.json does not exist.
/content/session1/test.json does not exist.
/content/session2/train.json does not exist.
/content/session2/test.json does not exist.
/content/session3/train.json does not exist.
/content/session3/test.json does not exist.
/content/session4/train.json does not exist.
/content/session4/test.json does not exist.
/content/session5/train.json does not exist.
/content/session5/test.json does not exist.


In [8]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session1/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/iemocap/session1/test.json",'r') as f:
    test_data = json.load(f)

In [9]:
   train_data

{'Ses02F_impro03_F000': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': '/content/iemocap/IEMOCAP_full_release/Session2/sentences/wav/Ses0

In [ ]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [ ]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1241
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1241
    })
})

In [ ]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap/
    example['wav'] = os.path.join('/content/iemocap/', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

In [ ]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [ ]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/iemocap/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'